# Supermarket Loyalty

This is a project by DataCamp which provided the data.

International Essentials is an international supermarket chain.

Shoppers at their supermarkets can sign up for a loyalty program that provides rewards each year to customers based on their spending. The more you spend the bigger the rewards. 

The supermarket would like to be able to predict the likely amount customers in the program will spend, so they can estimate the cost of the rewards. 

This will help them to predict the likely profit at the end of the year.

## Data

The dataset contains records of customers for their last full year of the loyalty program.

| Column Name | Criteria                                                |
|-------------|---------------------------------------------------------|
|customer_id | Unique identifier for the customer. </br>Missing values are not possible due to the database structure. |
|spend | Continuous. </br>The total spend of the customer in their last full year. This can be any positive value to two decimal places. </br>Missing values should be replaced with 0. |
|first_month | Continuous. </br>The amount spent by the customer in their first month of the year. This can be any positive value, rounded to two decimal places. </br>Missing values should be replaced with 0. |
| items_in_first_month | Discrete. </br>The number of items purchased in the first month. Any integer value greater than or equal to zero. </br>Missing values should be replaced by 0. |  
| region | Nominal. </br>The geographic region that the customer is based in. One of four values Americas, Asia/Pacific, Europe, Middle East/Africa. </br>Missing values should be replaced with "Unknown". |
| loyalty_years | Oridinal. </br>The number of years the customer has been a part of the loyalty program. One of five ordered categories, '0-1', '1-3', '3-5', '5-10', '10+'. </br>Missing values should be replaced with '0-1'.|
| joining_month | Nominal. </br>The month the customer joined the loyalty program. One of 12 values "Jan", "Feb", "Mar", "Apr", etc. </br>Missing values should be replaced with "Unknown".|
| promotion | Nominal. </br>Did the customer join the loyalty program as part of a promotion? Either 'Yes' or 'No'. </br>Missing values should be replaced with 'No'.|


In [308]:
# Import pandas
import pandas as pd 

# Load data
df = pd.read_csv('loyalty.csv')

df.head()

,customer_id,spend,first_month,items_in_first_month,region,loyalty_years,joining_month,promotion
0,1,132.68,15.3,5,Asia/Pacific,5-10,Nov,No
1,2,106.45,16.2,14,Asia/Pacific,0-1,Feb,Yes
2,3,123.16,25.26,7,Middle East/Africa,10+,Dec,Yes
3,4,130.60,24.74,8,Middle East/Africa,3-5,Apr,No
4,5,130.41,25.59,8,Middle East/Africa,3-5,Apr,Yes


# Task 1

Before you fit any models, you will need to make sure the data is clean. 

The table below shows what the data should look like. 

Create a cleaned version of the dataframe. 

 - You should start with the data in the file "loyalty.csv". 

 - Your output should be a dataframe named `clean_data`. 

 - All column names and values should match the table below.

| Column Name | Criteria                                                |
|-------------|---------------------------------------------------------|
|customer_id | Unique identifier for the customer. </br>Missing values are not possible due to the database structure. |
|spend | Continuous. </br>The total spend of the customer in their last full year. This can be any positive value to two decimal places. </br>Missing values should be replaced with 0. |
|first_month | Continuous. </br>The amount spent by the customer in their first month of the year. This can be any positive value, rounded to two decimal places. </br>Missing values should be replaced with 0. |
| items_in_first_month | Discrete. </br>The number of items purchased in the first month. Any integer value greater than or equal to zero. </br>Missing values should be replaced by 0. |  
| region | Nominal. </br>The geographic region that the customer is based in. One of four values Americas, Asia/Pacific, Europe, Middle East/Africa. </br>Missing values should be replaced with "Unknown". |
| loyalty_years | Oridinal. </br>The number of years the customer has been a part of the loyalty program. One of five ordered categories, '0-1', '1-3', '3-5', '5-10', '10+'. </br>Missing values should be replaced with '0-1'.|
| joining_month | Nominal. </br>The month the customer joined the loyalty program. One of 12 values "Jan", "Feb", "Mar", "Apr", etc. </br>Missing values should be replaced with "Unknown".|
| promotion | Nominal. </br>Did the customer join the loyalty program as part of a promotion? Either 'Yes' or 'No'. </br>Missing values should be replaced with 'No'.|

In [309]:
# Use this cell to write your code for Task 1

def transform_data(data):
    # Check for missing data in 'customer_id'
    if data['customer_id'].isna().sum() != 0:
        print("There is missing data in 'customer_id'. This should not be possible due to the database structure.")

    # Replace missing values with specified values
    data.fillna({'spend': 0, 'first_month': 0, 'items_in_first_month': 0, 'region': 'Unknown', 'loyalty_years': '0-1', 'joining_month': 'Unknown', 'promotion': 'No'}, inplace=True)
    
    # Replace non-numeric values in 'first_month' with 0
    data['first_month'] = pd.to_numeric(data['first_month'], errors='coerce').fillna(0)
    
    # Convert data types
    data['spend'] = data['spend'].astype(float)
    data['items_in_first_month'] = data['items_in_first_month'].astype(int)

    # Clean categorical and text data by manipulating strings
    data['region'] = data['region'].str.strip().str.title()
    data['joining_month'] = data['joining_month'].str.strip().str.title()
    data['promotion'] = data['promotion'].str.strip().str.title()

    # Convert 'Yes' and 'No' to uppercase
    data['promotion'] = data['promotion'].str.upper()

    # Replace variations of 'Yes' and 'No' with standardized values
    data['promotion'] = data['promotion'].replace(['YES', 'NO'], ['Yes', 'No'])

    # Identify and replace missing values
    data = data.replace(['','.','--','-'], pd.NA)
    data = data.fillna({'spend': 0, 'first_month': 0, 'items_in_first_month': 0, 'region': 'Unknown', 'loyalty_years': '0-1', 'joining_month': 'Unknown', 'promotion': 'No'})

    return data

clean_data = transform_data(df)

clean_data.head()

,customer_id,spend,first_month,items_in_first_month,region,loyalty_years,joining_month,promotion
0,1,132.68,15.30,5,Asia/Pacific,5-10,Nov,No
1,2,106.45,16.20,14,Asia/Pacific,0-1,Feb,Yes
2,3,123.16,25.26,7,Middle East/Africa,10+,Dec,Yes
3,4,130.60,24.74,8,Middle East/Africa,3-5,Apr,No
4,5,130.41,25.59,8,Middle East/Africa,3-5,Apr,Yes


# Task 2 

The team at International Essentials have told you that they have always believed that the number of years in the loyalty scheme is the biggest driver of spend. 

Producing a table showing the difference in the average spend by number of years in the loyalty programme along with the variance to investigate this question for the team.

 - You should start with the data in the file 'loyalty.csv'.

 - Your output should be a data frame named `spend_by_years`. 

 - It should include the three columns `loyalty_years`, `avg_spend`, `var_spend`. 

 - Your answers should be rounded to 2 decimal places.   

In [310]:
# Use this cell to write your code for Task 2

# Group the data according to 'loyalty_years'
spend_by_years = clean_data.groupby('loyalty_years')['spend'].agg({'mean', 'var'}).reset_index()

# Rename columns
spend_by_years.rename(columns = {'var': 'var_spend', 'mean': 'avg_spend'}, inplace= True)

# Round the data as requested
spend_by_years['var_spend'] = spend_by_years['var_spend'].round(2)
spend_by_years['avg_spend'] = spend_by_years['avg_spend'].round(2)

spend_by_years

,loyalty_years,var_spend,avg_spend
0,0-1,9.30,110.56
1,1-3,9.65,129.31
2,10+,16.72,117.41
3,3-5,11.09,124.55
4,5-10,14.10,135.15


# Task 3

Fit a baseline model to predict the spend over the year for each customer.

 1. Fit your model using the data contained in “train.csv” </br></br>

 2. Use “test.csv” to predict new values based on your model. You must return a dataframe named `base_result`, that includes `customer_id` and `spend`. The `spend` column must be your predicted values.

In [311]:
# Use this cell to write your code for Task 3
from sklearn.linear_model import LinearRegression

# Import data
train_data = pd.read_csv('train.csv')
test_data = pd.read_csv('test.csv')

# Encode data
train_data = pd.get_dummies(train_data, columns=['region', 'loyalty_years', 'joining_month', 'promotion'])
test_data = pd.get_dummies(test_data, columns=['region', 'loyalty_years', 'joining_month', 'promotion'])

# Create model
model1 = LinearRegression()

# Fit model
model1.fit(train_data.drop('spend', axis=1), train_data['spend'])

# Make predictions on the test data
predicted_spend = model1.predict(test_data)

# Create final dataframe
base_result = pd.DataFrame({'customer_id': test_data['customer_id'], 'spend':predicted_spend})

base_result

,customer_id,spend
0,5,140.711949
1,7,148.742502
2,16,140.822343
3,17,150.659149
4,19,153.641091
...,...,...
245,1216,134.859485
246,1225,148.531257
247,1231,136.955810
248,1242,129.873862


# Task 4

Fit a comparison model to predict the spend over the year for each customer.

 1. Fit your model using the data contained in “train.csv” </br></br>

 2. Use “test.csv” to predict new values based on your model. You must return a dataframe named `compare_result`, that includes `customer_id` and `spend`. The `spend` column must be your predicted values.

In [312]:
# Use this cell to write your code for Task 4

from sklearn.ensemble import RandomForestRegressor

# Import data
train_data = pd.read_csv('train.csv')
test_data = pd.read_csv('test.csv')

# Encode data
train_data = pd.get_dummies(train_data, columns=['region', 'loyalty_years', 'joining_month', 'promotion'])
test_data = pd.get_dummies(test_data, columns=['region', 'loyalty_years', 'joining_month', 'promotion'])

# Create model
model2 = RandomForestRegressor()

# Fit model
model2.fit(train_data.drop('spend', axis=1), train_data['spend'])

# Make predictions on the test data
predicted_spend2 = model2.predict(test_data)

# Create final dataframe for comparison
compare_result = pd.DataFrame({'customer_id': test_data['customer_id'], 'spend': predicted_spend2})

compare_result

,customer_id,spend
0,5,140.5423
1,7,148.9406
2,16,140.9327
3,17,150.8311
4,19,153.6300
...,...,...
245,1216,134.8105
246,1225,148.5203
247,1231,137.0556
248,1242,129.8913
